## Cell 1: Imports and Setup


In [ ]:
## Cell 1: Imports and Setup
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import random
from datetime import datetime
from tensorflow.keras.preprocessing import image

print("TensorFlow version:", tf.__version__)
print("Starting Grad-CAM analysis...")


## Cell 2: Grad-CAM Visualization Functions


In [ ]:
def superimpose_heatmap_tf(original_img, heatmap, alpha=0.4):
    """Superimpose heatmap on original image using only TensorFlow operations."""
    # Convert heatmap to RGB using a colormap (simulate jet colormap)
    heatmap = np.uint8(255 * heatmap)
    
    # Create a simple colormap (red for high values, blue for low)
    heatmap_rgb = np.zeros((*heatmap.shape, 3), dtype=np.uint8)
    
    # Simple colormap: red -> yellow -> green -> blue
    heatmap_rgb[heatmap > 192, 0] = 255  # Red
    heatmap_rgb[heatmap > 192, 1] = heatmap[heatmap > 192] - 192
    heatmap_rgb[heatmap > 192, 2] = 0
    
    heatmap_rgb[(heatmap > 128) & (heatmap <= 192), 0] = 255
    heatmap_rgb[(heatmap > 128) & (heatmap <= 192), 1] = 255
    heatmap_rgb[(heatmap > 128) & (heatmap <= 192), 2] = 0
    
    heatmap_rgb[(heatmap > 64) & (heatmap <= 128), 0] = (heatmap[(heatmap > 64) & (heatmap <= 128)] - 64) * 4
    heatmap_rgb[(heatmap > 64) & (heatmap <= 128), 1] = 255
    heatmap_rgb[(heatmap > 64) & (heatmap <= 128), 2] = 0
    
    heatmap_rgb[heatmap <= 64, 0] = 0
    heatmap_rgb[heatmap <= 64, 1] = heatmap[heatmap <= 64] * 4
    heatmap_rgb[heatmap <= 64, 2] = 255
    
    # Resize heatmap to match original image using TensorFlow
    heatmap_rgb = tf.image.resize(
        heatmap_rgb[np.newaxis, ...], 
        [original_img.shape[0], original_img.shape[1]]
    ).numpy()[0].astype(np.uint8)
    
    # Superimpose the heatmap on original image
    superimposed_img = heatmap_rgb * alpha + original_img * (1 - alpha)
    superimposed_img = np.clip(superimposed_img, 0, 255).astype(np.uint8)
    
    return superimposed_img

def run_gradcam(model, img_path, target_size=(224, 224)):
    """Run Grad-CAM analysis on a single image"""
    try:
        # Load and preprocess image
        img = tf.keras.utils.load_img(img_path, target_size=target_size)
        img_array = tf.keras.utils.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0
        
        # Get prediction
        preds = model.predict(img_array, verbose=0)
        predicted_class = "RESTRICTED" if preds[0][0] > 0.5 else "UNRESTRICTED"
        confidence = preds[0][0] if preds[0][0] > 0.5 else 1 - preds[0][0]
        
        # Get base model and last conv layer
        base_model = model.get_layer('inception_v3')
        last_conv_layer = base_model.get_layer('mixed10')
        
        # Create model for Grad-CAM
        cam_model = tf.keras.models.Model(
            base_model.input,
            [last_conv_layer.output, base_model.output]
        )
        
        # Get augmented image (pass through data augmentation)
        augmented_img = model.layers[1](img_array, training=False)
        
        # Compute Grad-CAM
        with tf.GradientTape() as tape:
            conv_output, base_pred = cam_model(augmented_img)
            output_value = base_pred[0]
            
        grads = tape.gradient(output_value, conv_output)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        
        # Generate heatmap
        conv_output = conv_output[0]
        heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_output), axis=-1)
        heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
        heatmap = tf.image.resize(heatmap[..., tf.newaxis], target_size).numpy().squeeze()
        
        # Prepare images for display
        original_img = (img_array[0] * 255).astype(np.uint8)
        superimposed_img = superimpose_heatmap_tf(original_img, heatmap)
        
        # Display results
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        axes[0].imshow(original_img / 255.0)
        axes[0].set_title(f'Original\\n{predicted_class} ({confidence:.2%})')
        axes[0].axis('off')
        
        axes[1].imshow(heatmap, cmap='jet')
        axes[1].set_title('Grad-CAM Heatmap')
        axes[1].axis('off')
        
        axes[2].imshow(superimposed_img / 255.0)
        axes[2].set_title('Superimposed')
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        return heatmap, superimposed_img, predicted_class, confidence
        
    except Exception as e:
        print(f"❌ Grad-CAM failed for {img_path}: {e}")
        return None, None, None, None

## Cell 3: Load Trained Model


In [ ]:
# Load your best InceptionV3 model
model_path = "models/InceptionV3_best.keras"

if os.path.exists(model_path):
    print("Loading trained model...")
    model = tf.keras.models.load_model(model_path)
    print("✅ Model loaded successfully!")
else:
    print("❌ Model file not found. Please check the path.")
    print("Expected path:", model_path)
    print("Current working directory:", os.getcwd())
    model = None

##  Cell 4: Find Test Images


In [ ]:
# Find test images from test/restricted and test/unrestricted folders
test_images = []
search_paths = [
    "test/restricted/*.jpg",
    "test/unrestricted/*.jpg"
]

for path_pattern in search_paths:
    found_images = glob.glob(path_pattern)
    test_images.extend(found_images)

# Remove duplicates and display
test_images = list(set(test_images))

if test_images:
    print("✅ Found test images:")
    for img_path in test_images[:10]:
        print(f"  📄 {img_path}")
    if len(test_images) > 10:
        print(f"  ... and {len(test_images) - 10} more")
else:
    print("❌ No test images found. Please check the paths.")
    print("Searched in:", search_paths)

## Cell 6: Run Grad-CAM on ALL Test Images


In [ ]:
if test_images and model is not None:
    print(f"🔍 Running Grad-CAM on ALL {len(test_images)} test images...")
    
    # Run Grad-CAM on all images (this will take some time)
    for i, img_path in enumerate(test_images):
        class_label = "RESTRICTED" if "restricted" in img_path else "UNRESTRICTED"
        print(f"\n{'='*60}")
        print(f"{i+1}/{len(test_images)}: {class_label} - {os.path.basename(img_path)}")
        print(f"{'='*60}")
        
        heatmap, superimposed, pred_class, confidence = run_gradcam(model, img_path)
        
        if heatmap is not None:
            print(f"✅ Analysis complete: {pred_class} ({confidence:.2%} confidence)")
        else:
            print("❌ Analysis failed")
    
    print(f"\n🎉 Grad-CAM analysis complete on ALL {len(test_images)} test images!")
else:
    print("❌ Cannot run Grad-CAM. Check if model and test images are available.")

## Cell 7: Save Grad-CAM Results for ALL Images


In [ ]:
def save_gradcam_results(model, img_path, output_dir="gradcam_results", target_size=(224, 224)):
    """Run Grad-CAM and save all results"""
    try:
        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        # Get image name for saving
        img_name = os.path.basename(img_path).split('.')[0]
        class_folder = "restricted" if "restricted" in img_path else "unrestricted"
        class_dir = os.path.join(output_dir, class_folder)
        os.makedirs(class_dir, exist_ok=True)
        
        # Load and preprocess image
        img = tf.keras.utils.load_img(img_path, target_size=target_size)
        img_array = tf.keras.utils.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0
        
        # Get prediction
        preds = model.predict(img_array, verbose=0)
        predicted_class = "RESTRICTED" if preds[0][0] > 0.5 else "UNRESTRICTED"
        confidence = preds[0][0] if preds[0][0] > 0.5 else 1 - preds[0][0]
        
        # Get base model and last conv layer
        base_model = model.get_layer('inception_v3')
        last_conv_layer = base_model.get_layer('mixed10')
        
        # Create model for Grad-CAM
        cam_model = tf.keras.models.Model(
            base_model.input,
            [last_conv_layer.output, base_model.output]
        )
        
        # Get augmented image (pass through data augmentation)
        augmented_img = model.layers[1](img_array, training=False)
        
        # Compute Grad-CAM
        with tf.GradientTape() as tape:
            conv_output, base_pred = cam_model(augmented_img)
            output_value = base_pred[0]
            
        grads = tape.gradient(output_value, conv_output)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        
        # Generate heatmap
        conv_output = conv_output[0]
        heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_output), axis=-1)
        heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
        heatmap = tf.image.resize(heatmap[..., tf.newaxis], target_size).numpy().squeeze()
        
        # Prepare images for saving
        original_img = (img_array[0] * 255).astype(np.uint8)
        superimposed_img = superimpose_heatmap_tf(original_img, heatmap)
        
        # Save all images
        # 1. Original image
        original_path = os.path.join(class_dir, f"{img_name}_original.png")
        plt.imsave(original_path, original_img / 255.0)
        
        # 2. Heatmap only
        heatmap_path = os.path.join(class_dir, f"{img_name}_heatmap.png")
        plt.imsave(heatmap_path, heatmap, cmap='jet')
        
        # 3. Superimposed image
        superimposed_path = os.path.join(class_dir, f"{img_name}_superimposed.png")
        plt.imsave(superimposed_path, superimposed_img / 255.0)
        
        # 4. Combined comparison image
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        axes[0].imshow(original_img / 255.0)
        axes[0].set_title(f'Original\\n{predicted_class} ({confidence:.2%})')
        axes[0].axis('off')
        
        axes[1].imshow(heatmap, cmap='jet')
        axes[1].set_title('Heatmap')
        axes[1].axis('off')
        
        axes[2].imshow(superimposed_img / 255.0)
        axes[2].set_title('Superimposed')
        axes[2].axis('off')
        
        combined_path = os.path.join(class_dir, f"{img_name}_combined.png")
        plt.savefig(combined_path, bbox_inches='tight', dpi=300)
        plt.close()
        
        return True
        
    except Exception as e:
        print(f"❌ Error saving {img_path}: {e}")
        return False

# Save results for ALL test images
if test_images and model is not None:
    print(f"\n💾 Saving Grad-CAM results for ALL {len(test_images)} test images...")
    print("This will take some time (a few minutes)...")
    
    success_count = 0
    for i, img_path in enumerate(test_images):
        if i % 10 == 0:  # Print progress every 10 images
            print(f"Processing image {i+1}/{len(test_images)}...")
        
        if save_gradcam_results(model, img_path):
            success_count += 1
    
    print(f"\n🎉 Successfully saved {success_count}/{len(test_images)} images")
    print(f"📁 Results saved in 'gradcam_results/' folder")

##  Cell 8: Create Summary Report for ALL Images


In [ ]:
def create_summary_report(test_images, output_dir="gradcam_results"):
    """Create HTML summary report for all images"""
    try:
        report_path = os.path.join(output_dir, "gradcam_summary.html")
        
        with open(report_path, 'w') as f:
            f.write(f'''
            <!DOCTYPE html>
            <html>
            <head>
                <title>Grad-CAM Results Summary - ALL Test Images</title>
                <style>
                    body {{ font-family: Arial, sans-serif; margin: 20px; }}
                    .header {{ text-align: center; margin-bottom: 30px; }}
                    .image-grid {{ display: grid; grid-template-columns: repeat(2, 1fr); gap: 20px; }}
                    .image-item {{ border: 1px solid #ddd; padding: 10px; text-align: center; }}
                    .image-item img {{ max-width: 100%; height: auto; }}
                    .class-header {{ margin-top: 40px; color: #333; border-bottom: 2px solid #333; }}
                    .stats {{ background: #f5f5f5; padding: 15px; border-radius: 5px; margin: 20px 0; }}
                </style>
            </head>
            <body>
                <div class="header">
                    <h1>🎯 Grad-CAM Analysis Summary - ALL Test Images</h1>
                    <p>Generated on: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}</p>
                    <div class="stats">
                        <p><strong>Total images analyzed:</strong> {len(test_images)}</p>
                        <p><strong>Restricted breeds:</strong> {len([img for img in test_images if "restricted" in img])}</p>
                        <p><strong>Unrestricted breeds:</strong> {len([img for img in test_images if "unrestricted" in img])}</p>
                    </div>
                </div>
            ''')
            
            # Add restricted images
            restricted_imgs = [img for img in test_images if "restricted" in img]
            f.write(f'<h2 class="class-header">🚫 RESTRICTED Breeds ({len(restricted_imgs)} images)</h2>')
            f.write('<div class="image-grid">')
            for img_path in restricted_imgs:
                img_name = os.path.basename(img_path).split('.')[0]
                f.write(f'''
                <div class="image-item">
                    <h3>{img_name}</h3>
                    <img src="restricted/{img_name}_combined.png" alt="{img_name}">
                    <p>
                        <a href="restricted/{img_name}_original.png">Original</a> | 
                        <a href="restricted/{img_name}_heatmap.png">Heatmap</a> | 
                        <a href="restricted/{img_name}_superimposed.png">Overlay</a>
                    </p>
                </div>
                ''')
            f.write('</div>')
            
            # Add unrestricted images
            unrestricted_imgs = [img for img in test_images if "unrestricted" in img]
            f.write(f'<h2 class="class-header">✅ UNRESTRICTED Breeds ({len(unrestricted_imgs)} images)</h2>')
            f.write('<div class="image-grid">')
            for img_path in unrestricted_imgs:
                img_name = os.path.basename(img_path).split('.')[0]
                f.write(f'''
                <div class="image-item">
                    <h3>{img_name}</h3>
                    <img src="unrestricted/{img_name}_combined.png" alt="{img_name}">
                    <p>
                        <a href="unrestricted/{img_name}_original.png">Original</a> | 
                        <a href="unrestricted/{img_name}_heatmap.png">Heatmap</a> | 
                        <a href="unrestricted/{img_name}_superimposed.png">Overlay</a>
                    </p>
                </div>
                ''')
            f.write('</div></body></html>')
        
        print(f"📊 Summary report created: {report_path}")
        print("📍 Open 'gradcam_results/gradcam_summary.html' in your browser to view ALL results!")
        
    except Exception as e:
        print(f"❌ Error creating summary report: {e}")

# Create summary report for ALL images
if test_images:
    create_summary_report(test_images)

 ##  Cell 9: Performance Notes


In [ ]:
print(f"\n{'='*60}")
print("📊 PERFORMANCE NOTES")
print(f"{'='*60}")
print(f"Total test images: {len(test_images)}")
print("Estimated processing time:")
print(f"  - ~2-3 seconds per image")
print(f"  - Total: ~{len(test_images)*2}-{len(test_images)*3} seconds")
print(f"  - (~{len(test_images)*2//60}-{len(test_images)*3//60} minutes)")
print("\n💡 Tips:")
print("  - Let it run overnight if you have many images")
print("  - Results will be saved incrementally")
print("  - You can stop and restart anytime")
print("  - All images will be organized in gradcam_results/ folder")